### Week 3: How do we sequence antibiotics?

The following problem asks you to find the translation of an RNA string into an amino acid string.

**Protein Translation Problem**: *Translate an RNA string into an amino acid string.*

* **Input**: An RNA string *Pattern* and the array *GeneticCode*.
* **Output**: The translation of *Pattern* into an amino acid string *Peptide*.

In [17]:
from helpers import *
from display import *

def translate(seq: str) -> str:
    return ''.join(CODON_TABLE[codon] for codon in chunks(seq))

translate('AUGGCCAUGGCGCCCAGAACUGAGAUCAAUAGUACCCGUAUUAACGGGUGA')

'MAMAPRTEINSTRING'

We say that a DNA string *Pattern* **encodes** an amino acid string *Peptide* if the RNA string transcribed from either *Pattern* or its reverse complement *Pattern* translates into *Peptide*. For example, the DNA string `GAAACT` is transcribed into `GAAACU` and translated into `ET`. The reverse complement of this DNA string, `AGTTTC`, is transcribed into `AGUUUC` and translated into SF. Thus, `GAAACT` encodes both `ET` and `SF`.

**Peptide Encoding Problem**: *Find substrings of a genome encoding a given amino acid sequence.*

* **Input**: A DNA string *Text*, an amino acid string *Peptide*, and the array *GeneticCode*.
* **Output**: All substrings of *Text* encoding *Peptide* (if any such substrings exist).

In [12]:
def peptide_encoding_problem(dna: str, peptide: str) -> list[str]:
    fragments = []
    for frag in window(dna, len(peptide)*3):
        if (peptide == DNA_to_peptide(frag)) or (peptide == DNA_to_peptide(reverse_comp(frag))):
            fragments.append(frag)
    return fragments

Text = 'ATGGCCATGGCCCCCAGAACTGAGATCAATAGTACCCGTATTAACGGGTGA'
Peptide = 'MA'

print_list(peptide_encoding_problem(Text, Peptide), sep='\n')

ATGGCC
GGCCAT
ATGGCC


In [13]:
with open('dataset_96_7.txt', 'r') as f:
    Text = f.readline().strip()
    Peptide = f.readline().strip()
    print_list(peptide_encoding_problem(Text, Peptide), sep='\n')

CTTAAACCGGACGTCTAAGTACGGGTATAC
GTGTATCCTTACCTAGACGTCAGGTTTAAA
TTTGAAACGTACATCCAGATAGGGATATAC
GTCTACCCTTATTTAGACGTCCGCTTCAAA
GTATACCCGTATTTGGACGTTCGCTTCAAG
CTTAAATCGAACATCCAAATAGGGATAAAC
GTGTATCCCTACCTCGATGTGCGGTTTAAG
GTATATCCCTATCTCGATGTCCGATTCAAG
GTGTACCCCTATTTAGATGTGCGCTTTAAG
CTTAAAGCGAACGTCTAAGTAGGGGTATAC
CTTGAAACGTACATCTAGGTAAGGATAGAC
CTTGAAGCGTACGTCAAGATACGGATAAAC
TTTGAAACGAACGTCGAGGTATGGATATAC
TTTGAATCGGACATCTAAATAAGGATACAC
TTTGAACCGGACGTCCAGATAGGGATAAAC
CTTAAATCTAACATCTAAGTATGGGTAAAC
GTGTATCCGTATCTAGACGTGAGATTCAAA
GTATATCCATACTTGGATGTACGGTTTAAA
CTTGAAGCGTACGTCCAGGTAGGGGTATAC
CTTGAACCTTACATCTAAATACGGATATAC


Solve the Peptide Encoding Problem for *Bacillus brevis* and Tyrocidine B1 (`Val-Lys-Leu-Phe-Pro-Trp-Phe-Asn-Gln-Tyr``). How many starting positions in *Bacillus brevis* encode this peptide?

In [14]:
with open('Bacillus_brevis.txt', 'r') as f:
    genome = f.read().strip().replace('\n', '')
    tyrocidine_b1 = 'VKLFPWFNQY'
    print(len(peptide_encoding_problem(genome, tyrocidine_b1)))

0


The **theoretical spectrum** of a cyclic peptide *Peptide*, denoted *Cyclospectrum(Peptide)*, is the collection of all of the masses of its subpeptides, in addition to the mass 0 and the mass of the entire peptide, with masses ordered from smallest to largest. We will assume that the theoretical spectrum can contain duplicate elements, as is the case for NQEL (shown below), where NQ and EL have the same mass.

![duplicate elements](http://bioinformaticsalgorithms.com/images/Antibiotics/duplicate_elements.png)

**Generating Theoretical Spectrum Problem**: *Generate the theoretical spectrum of a cyclic peptide.*

* **Input**: An amino acid string *Peptide*.
* **Output**: *Cyclospectrum(Peptide)*.

In [26]:
INTEGER_MASSES = {}
with open('integer_mass_table.txt', 'r') as f:
    masses = [line.strip().split() for line in f]
    INTEGER_MASSES = { mass[0]: int(mass[1]) for mass in masses }

def linear_spectrum(peptide: str) -> list[int]:
    prefix_mass = []
    for i in range(0, len(peptide)+1):
        # Sum up the masses of each amino acid in this subpeptide
        prefix_mass.append(sum(INTEGER_MASSES[aa] for aa in peptide[:i]))
    linear_spectrum = [0]
    for i in range(len(prefix_mass)):
        for j in range(i+1, len(prefix_mass)):
            linear_spectrum.append(prefix_mass[j] - prefix_mass[i])
    return sorted(linear_spectrum)

peptide_seq = 'NQEL'
print_list(linear_spectrum(peptide_seq))

0 113 114 128 129 242 242 257 370 371 484


If *Peptide* represents a cyclic peptide instead, then the masses in its theoretical spectrum can be divided into those found by *LinearSpectrum* and those corresponding to subpeptides wrapping around the end of *Peptide*. Furthermore, each such subpeptide has mass equal to the difference between *Mass(Peptide)* and a subpeptide mass identified by *LinearSpectrum*.

Thus, we can generate a cyclic spectrum by making only a small modification to *LinearSpectrum*.

In [22]:
def cyclic_spectrum(peptide: str) -> list[int]:
    prefix_mass = []
    for i in range(0, len(peptide)+1):
        prefix_mass.append(sum(INTEGER_MASSES[aa] for aa in peptide[:i]))
    peptide_mass = prefix_mass[-1]
    cyclic_spectrum = [0]
    for i in range(len(prefix_mass)):
        for j in range(i+1, len(prefix_mass)):
            cyclic_spectrum.append(prefix_mass[j] - prefix_mass[i])
            if (i > 0) and (j < len(peptide)):
                cyclic_spectrum.append(peptide_mass - cyclic_spectrum[-1])
    return sorted(cyclic_spectrum)

peptide_seq = 'LEQN'
print_list(cyclic_spectrum(peptide_seq))

0 113 114 128 129 227 242 242 257 355 356 370 371 484
